In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -uq "/content/drive/My Drive/archive" -d "/content/drive/My Drive/archive"

In [ ]:
#Alternate Way to import dataset directly from Kaggle
'''import os
os.environ['KAGGLE_USERNAME'] = 'anshikabisht'
os.environ['KAGGLE_KEY'] = 'b2c6365189eaa138bbeef72e68bc7dd5' '''

In [ ]:
'''! kaggle datasets download -d prithwirajmitra/covid-face-mask-detection-dataset '''

100% 207M/207M [00:01<00:00, 152MB/s]
100% 207M/207M [00:01<00:00, 124MB/s]


In [ ]:
'''! unzip covid-face-mask-detection-dataset.zip '''

Archive:  covid-face-mask-detection-dataset.zip
  inflating: New Masks Dataset/Test/Mask/2070.jpg  
  inflating: New Masks Dataset/Test/Mask/2072.jpg  
  inflating: New Masks Dataset/Test/Mask/2083.jpg  
  inflating: New Masks Dataset/Test/Mask/2085.jpg  
  inflating: New Masks Dataset/Test/Mask/2086.jpg  
  inflating: New Masks Dataset/Test/Mask/2110.jpg  
  inflating: New Masks Dataset/Test/Mask/2113.jpg  
  inflating: New Masks Dataset/Test/Mask/2114.jpeg  
  inflating: New Masks Dataset/Test/Mask/2130.jpg  
  inflating: New Masks Dataset/Test/Mask/2132.jpg  
  inflating: New Masks Dataset/Test/Mask/2135.jpg  
  inflating: New Masks Dataset/Test/Mask/2154.png  
  inflating: New Masks Dataset/Test/Mask/2158.png  
  inflating: New Masks Dataset/Test/Mask/2159.png  
  inflating: New Masks Dataset/Test/Mask/2160.png  
  inflating: New Masks Dataset/Test/Mask/2170.png  
  inflating: New Masks Dataset/Test/Mask/2172.png  
  inflating: New Masks Dataset/Test/Mask/2173.png  
  inflating: Ne

In [ ]:
import os
main_dir = "/content/drive/My Drive/archive/New Masks Dataset"
train_dir = os.path.join(main_dir, 'Train')
test_dir = os.path.join(main_dir, 'Test')
valid_dir = os.path.join(main_dir, 'Validation')

train_mask_dir = os.path.join(train_dir, 'Mask')
train_non_mask_dir = os.path.join(train_dir, 'Non Mask')

In [ ]:
print(train_dir)

/content/drive/My Drive/archive/New Masks Dataset/Train


In [ ]:
train_mask_names = os.listdir(train_mask_dir)
print(train_mask_names[:10])

train_nomask_names = os.listdir(train_non_mask_dir)
print(train_nomask_names[:10])

['0042.jpg', '0027.jpg', '0022.jpg', '0040.jpg', '0003.jpg', '0029.jpg', '0037.jpg', '0006.jpg', '0018.jpg', '0019.jpg']
['115.jpg', '143.jpg', '151.jpg', '141.jpg', '152.jpg', '14.jpg', '122.jpg', '133.jpg', '125.jpg', '138.jpg']


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, zoom_range = 0.2, rotation_range = 40, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size = (150,150), batch_size = 32, class_mode = 'binary')

test_generator = train_datagen.flow_from_directory(test_dir, target_size = (150,150), batch_size = 32, class_mode = 'binary')

valid_generator = train_datagen.flow_from_directory(valid_dir, target_size = (150,150), batch_size = 32, class_mode = 'binary')

Found 600 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Found 306 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'Mask': 0, 'Non Mask': 1}

In [ ]:
train_generator.image_shape

(150, 150, 3)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding = 'SAME', activation = 'relu', input_shape = (150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3,3), padding = 'SAME', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 37, 37, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 87616)             0

In [ ]:
model.compile(Adam(lr = 0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
history = model.fit(train_generator, epochs = 30, validation_data = valid_generator)

Epoch 1/30
19/19 [==============================] - 423s 23s/step - loss: 3.0293 - accuracy: 0.4717 - val_loss: 0.6922 - val_accuracy: 0.5000
Epoch 2/30
19/19 [==============================] - 37s 2s/step - loss: 0.6299 - accuracy: 0.6167 - val_loss: 0.6475 - val_accuracy: 0.7418
Epoch 3/30
19/19 [==============================] - 37s 2s/step - loss: 0.4884 - accuracy: 0.7917 - val_loss: 0.5479 - val_accuracy: 0.8105
Epoch 4/30
19/19 [==============================] - 37s 2s/step - loss: 0.3968 - accuracy: 0.8417 - val_loss: 0.5461 - val_accuracy: 0.7876
Epoch 5/30
19/19 [==============================] - 37s 2s/step - loss: 0.3547 - accuracy: 0.8867 - val_loss: 0.3969 - val_accuracy: 0.8529
Epoch 6/30
19/19 [==============================] - 37s 2s/step - loss: 0.2912 - accuracy: 0.8900 - val_loss: 0.3892 - val_accuracy: 0.8366
Epoch 7/30
19/19 [==============================] - 36s 2s/step - loss: 0.2817 - accuracy: 0.9017 - val_loss: 0.3166 - val_accuracy: 0.8922
Epoch 8/30
19/19 [

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print('test_acc : {} test_loss : {}'.format(test_acc, test_loss))

4/4 [==============================] - 42s 14s/step - loss: 0.2297 - accuracy: 0.9100
test_acc : 0.9100000262260437 test_loss : 0.22967630624771118


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

fname = files.upload()
#print(fname)
f = list(fname.keys())[0]
img_path = '/content/' + f
img = image.load_img(img_path, target_size = (150, 150))
images = image.img_to_array(img)
images = np.expand_dims(images, axis = 0)
prediction = model.predict(images)
if prediction == 0:
  print('Mask')
else:
  print('No mask')

Saving real_01078.jpg to real_01078.jpg
No mask


In [ ]:
model.save('savemodel.h5')

In [ ]:
'''import numpy as np
from google.colab import files
from keras.preprocessing import image'''

#Alternate way to upload and check for multiple files at once

'''uploaded = files.upload()
for f in uploaded.keys():
  img_path = '/content/' + f
  img = image.load_img(img_path, target_size = (150, 150))
  images = image.img_to_array(img)
  images = np.expand_dims(images, axis = 0)
  prediction = model.predict(images)
  if prediction == 0:
    print(f, ': Mask')
  else:
    print(f, ': No mask') '''